In [6]:
import pandas as pd
import numpy as np
%config Completer.use_jedi = False

stk_off = pd.DataFrame({'Ticker': ['EWZ US 05/29/20 P27', 'HSI US 12/30/20 C24800', 'TLT US 06/19/20 C225', 'EWZ US 05/29/20 P29', 'XLE US 05/29/20 P30'],
                   'Instrument': ['EWZ', 'HSI', 'TLT', 'EWZ', 'EWZ'],
                   'Maturity Label': ['MAI 20 D29', 'DEC 20 D30', 'JUN 20 D19', 'MAY 20 D29', 'MAY 20 D29'],
                   'Strike': [27, 24800, 225, 29, 29],
                   'Payout': ['P', 'C', 'C', 'P', 'P'],
                   'Style': ['A', 'E', 'E', 'A', 'A'],
                   'Maturity': ['29/05/2020', '12/30/2020', '19/06/2020', '29/05/2020', '29/05/2020'],
                   'Market': ['US NYSE', 'US NYSE', 'HK HKSE', 'US NYSE', 'US NYSE']})

stk_off['Reduced_Ticker'] = stk_off['Ticker'].apply(lambda a :" ".join(a.split(" ", 2)[:2]))

In [7]:
market_ticker = stk_off[['Market','Reduced_Ticker']].groupby(['Market']).agg(list)
market_ticker['Reduced_Ticker'] = market_ticker['Reduced_Ticker'].apply(lambda x: pd.unique(x))
market_ticker = market_ticker.explode('Reduced_Ticker')
market_ticker = market_ticker.reset_index()

In [8]:
market_ticker

,Market,Reduced_Ticker
0,HK HKSE,TLT US
1,US NYSE,EWZ US
2,US NYSE,HSI US
3,US NYSE,XLE US


In [9]:
instrument_ticker = stk_off[['Instrument','Reduced_Ticker']].groupby(['Instrument']).agg(list)
instrument_ticker['Reduced_Ticker'] = instrument_ticker['Reduced_Ticker'].apply(lambda x: pd.unique(x))
instrument_ticker = instrument_ticker.explode('Reduced_Ticker')
instrument_ticker = instrument_ticker.reset_index()
instrument_ticker

,Instrument,Reduced_Ticker
0,EWZ,EWZ US
1,EWZ,XLE US
2,HSI,HSI US
3,TLT,TLT US


In [10]:
tickers = ['EWZ US 05/29/20 C31','HSI US 12/30/20 C24900']
tickers_reduced = [t.split()[0] for t in tickers]

selection_instrument = (instrument_ticker
             .explode('Reduced_Ticker')
             ['Reduced_Ticker'].apply(lambda x: x.split()[0] in tickers_reduced))

instrument = instrument_ticker.loc[selection_instrument].Instrument.to_list()
instrument

['EWZ', 'HSI']

In [11]:
market_ticker = market_ticker.reset_index()
selection_market = (market_ticker
                   .explode('Reduced_Ticker')
                   ['Reduced_Ticker'].apply(lambda x: x.split()[0] in tickers_reduced))


market = market_ticker.loc[selection_market].Market.to_list()
market

['US NYSE', 'US NYSE']

In [12]:
payout_strike = [t.split()[3] for t in tickers]
payout = [p[0] for p in payout_strike]
strike = [s[1:] for s in payout_strike]

In [13]:
import datetime
maturity = [datetime.datetime.strptime(x.split()[2], '%m/%d/%y').strftime('%d/%m/%Y') for x in tickers]
maturity

['29/05/2020', '30/12/2020']

In [14]:
import xlrd
maturities_usnyse = pd.read_excel('/home/gustavo/Desktop/Fut_Opt/Vencimentos/US NYSE.xlsx')
maturities_usnyse['Maturity'] = [datetime.datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S').strftime('%d/%m/%Y') 
                     for x in maturities_usnyse.Maturity]
maturities_usnyse

/home/gustavo/Desktop/Fut_Opt/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  


,Label,First Td,Maturity
0,MAI 20 D29,NaN,29/05/2020
1,DEC 20 D30,NaN,30/12/2020


In [15]:

selection_label = (maturities_usnyse.explode('Maturity')['Maturity'].apply(lambda x: x in maturity))

maturity_label = maturities_usnyse.loc[selection_label].Label.to_list()
maturity_label

['MAI 20 D29', 'DEC 20 D30']

In [11]:
df_to_upload = pd.DataFrame()

df_to_upload['Ticker'] = tickers
df_to_upload['Instrument'] = instrument
df_to_upload['Maturity Label'] = maturity_label
df_to_upload['Strike'] = strike
df_to_upload['Payout'] = payout
df_to_upload['Maturity'] = maturity
df_to_upload['Market'] = market
df_to_upload['']

df_to_upload

,Ticker,Instrument,Maturity Label,Strike,Payout,Maturity,Market
0,EWZ US 05/29/20 C31,EWZ,MAI 20 D29,31,C,29/05/2020,US NYSE
1,HSI US 12/30/20 C24900,HSI,DEC 20 D30,24900,C,30/12/2020,US NYSE


In [18]:
def style(df):
    df['Style'] = ''
    for x in df['Market']:
        if x == 'US NYSE':
            df['Style'] = 'A'
        else:
            df['Style'] = 'E'
    
    return df

In [19]:
df_to_upload = style(df_to_upload)
df_to_upload

,Ticker,Instrument,Maturity Label,Strike,Payout,Maturity,Market,Style
0,EWZ US 05/29/20 C31,EWZ,MAI 20 D29,31,C,29/05/2020,US NYSE,A
1,HSI US 12/30/20 C24900,HSI,DEC 20 D30,24900,C,30/12/2020,US NYSE,A
